In [233]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.svm import OneClassSVM
import numpy as np
from scipy import sparse

In [234]:
df_train = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\treino\r2_bo_train_statements.csv', sep=';')
df_train = df_train.drop(df_train[df_train.Polarity =='other'].index)
df_train.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,Maduro diz que Bolsonaro arrasta as forças mil...,r2_bo,0,yes,r2_bo_75,_1228475221147029504,2259,"[('maduro', 'ADJ'), ('diz', 'V'), ('que', 'KS'...",[],"{'bolsonaro': 1, 'brasil': 1, 'conflito': 1, '..."
1,"Prezada, Bozo é ignorante, e interpretou errad...",r2_bo,0,no,r2_bo_57,_1270354943518572545,4492,"[('prezada', 'PCP'), (',', 'PU'), ('bozo', 'N'...",[],"{'bozo': 1, 'transmissão': 1, 'coronavírus': 1..."
2,"se a china enviou o coronavírus pra cá, a gent...",r2_bo,0,no,r2_bo_474,_1264295371695697920,4587,"[('se', 'KS'), ('a', 'ART'), ('china', 'N'), (...",[],"{'coronavírus': 1, 'bolsonaro': 1, 'gente': 1,..."
3,Acho que para o tal Felipe Neto e outros Petis...,r2_bo,1,no,r2_bo_103,_1070318256907411456,51,"[('acho', 'V'), ('que', 'KS'), ('para', 'PREP'...",[],"{'bolsonaro': 1, 'tipo': 1, 'mente': 1, 'licen..."
4,Bolsonaro não incentiva a violência: A violênc...,r2_bo,1,no,r2_bo_93,_1031307320373010432,910,"[('bolsonaro', 'N'), ('não', 'ADV'), ('incenti...",[],"{'violência': 2, 'respostas': 1, 'bolsonaro': ..."
...,...,...,...,...,...,...,...,...,...,...
7055,Fica a vontade pra levar eleitores do Bolsonar...,r2_bo,0,no,r2_bo_637,_1206580142644113408,2099,"[('fica', 'V'), ('a', 'ART'), ('vontade', 'N')...",[],"{'eleitores': 1, 'bolsonaro': 1, 'vontade': 1,..."
7056,General detona Bolsonaro e critica uso de mili...,r2_bo,0,yes,r2_bo_75,_1217930731256713218,1648,"[('general', 'N'), ('detona', 'V'), ('bolsonar...",[],"{'general': 1, 'militares': 1, 'inss': 1, 'fil..."
7057,"o que significa isso???@ ,isso é gravíssimo!!!...",r2_bo,1,no,r2_bo_702,_1139401650114723840,1821,"[('@', 'NUM'), ('radardamidia', 'N'), (',', 'P...","['radardamidia', 'carlazambelli17', 'carlosbol...","{'homem': 1, 'bolsonaro': 1, 'radardamidia': 1..."
7058,Uma pessoa que assiste os debates e ainda tem ...,r2_bo,0,no,r2_bo_104,_1034606940566036482,3029,"[('uma', 'ART'), ('pessoa', 'N'), ('que', 'PRO...",[],"{'bolsonaro': 1, 'debates': 1, 'capacidade': 1..."


In [235]:
df_test = pd.read_csv(r'C:\Users\camil\OneDrive\Área de Trabalho\mestrado\Teste\r2_bo_test_statements.csv', sep=';')
df_test = df_test.drop(df_test[df_test.Polarity =='other'].index)
df_test.Polarity.replace(('for', 'against'), (1,0), inplace=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,Text,Target,Polarity,Factual,User_ID,Tweet_ID,Tweet_Seq,POS,Mentions,Nouns
0,ACABOU O CARINHO! vou excluir da minha vida qu...,r2_bo,0,no,r2_bo_72,_1270157032054763521,3894,"[('acabou', 'V'), ('o', 'ART'), ('carinho', 'N...",[],"{'apoio': 1, 'bolsonaro': 1, 'numero': 1, 'vid..."
1,Alô esquerdistas! Não é o presidente Bolsonaro...,r2_bo,1,no,r2_bo_752,_1257799483011796994,284,"[('alô', 'IN'), ('esquerdistas', 'N'), ('!', '...",[],"{'visto': 1, 'brasileiros': 1, 'manifestações'..."
2,A esquerda que só sabe falar mal do Bolsonaro ...,r2_bo,0,no,r2_bo_682,_1286673971237322754,3927,"[('a', 'ART'), ('esquerda', 'N'), ('que', 'PRO...",[],"{'esquerda': 7, 'q': 2, 'nd': 2, 'direita': 2,..."
3,"Sob Bolsonaro, Brasil é visto como um dos país...",r2_bo,0,yes,r2_bo_75,_1220375174014521351,1863,"[('sob', 'PREP'), ('bolsonaro', 'NPROP'), (','...",[],"{'países': 1, 'brasil': 1}"
4,bolsonaro fodase,r2_bo,0,no,r2_bo_4,_1316394060568035329,222,"[('@', 'NUM'), ('facts_bolsonaro', 'N'), ('fod...",['facts_bolsonaro'],"{'facts_bolsonaro': 1, 'fodase': 1}"
...,...,...,...,...,...,...,...,...,...,...
2349,"O gente, ninguém liga pro Bolsonaro não, nem p...",r2_bo,0,no,r2_bo_164,_1280659531517878272,4943,"[('o', 'ART'), ('gente', 'N'), (',', 'PU'), ('...",[],"{'ee': 1, 'gente': 1, 'família': 1}"
2350,"serio isso, o Bolsonaro mudou as cadeiras do p...",r2_bo,0,no,r2_bo_609,_1080957863437185024,2819,"[('serio', 'N'), ('isso', 'PROSUB'), (',', 'PU...",[],"{'serio': 2, 'palacio': 1, 'cadeiras': 1, 'bol..."
2351,"Sempre soube q Moro é progressista, mas confes...",r2_bo,1,no,r2_bo_723,_1253737658334969857,3686,"[('sempre', 'ADV'), ('soube', 'V'), ('q', 'ADJ...",[],"{'país': 2, 'tiro': 1, 'moro': 1, 'lado': 1, '..."
2352,PRESIDENTE bolsonaro A @ E O @ SÃO LIXO DA MÍD...,r2_bo,1,no,r2_bo_737,_1242643009382027264,3091,"[('>', 'PU'), ('>', 'PU'), ('presidente', 'N')...","['veramagalhaes', 'williamwaack_jg']","{'williamwaack_jg': 1, 'veramagalhaes': 1, 'pr..."


In [236]:
#carregando unlabeled data
df_u = pd.read_csv(r'C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/bolsonaro.csv')
#df_u = df.sample(frac=1).reset_index(drop=True)
df_u

,Unnamed: 0,Text
0,0,isso me lembra eu anto e bruna discutindo c o ...
1,1,bolsonaro fdp acabando c a maconha esse país t...
2,2,Bolsonaro 2018? — No dia das eleições eu ainda...
3,3,Eu amo o Bolsonaro
4,4,acabou os seiscentao do bolsonaro entao amanha...
...,...,...
31990,31990,Eu conto 0 items dessa lista no governo Bolson...
31991,31991,Não existe comparação nenhuma entre os últimos...
31992,31992,Sendo sincero: seria injusto até comparar a ca...
31993,31993,Eu vou ser sinceros com vcs: eu não acho que n...


In [237]:
rows_train = len(df_train.index)
rows_test = len(df_test.index)
rows_label=rows_train+rows_test

In [238]:
rows_unlabel = len(df_u.index)
rows_unlabel

31995

In [239]:
frames = [df_train.Text, df_test.Text, df_u.Text]
X = pd.concat(frames)
X

0        Maduro diz que Bolsonaro arrasta as forças mil...
1        Prezada, Bozo é ignorante, e interpretou errad...
2        se a china enviou o coronavírus pra cá, a gent...
3        Acho que para o tal Felipe Neto e outros Petis...
4        Bolsonaro não incentiva a violência: A violênc...
                               ...                        
31990    Eu conto 0 items dessa lista no governo Bolson...
31991    Não existe comparação nenhuma entre os últimos...
31992    Sendo sincero: seria injusto até comparar a ca...
31993    Eu vou ser sinceros com vcs: eu não acho que n...
31994    Esse vídeo da reunião dos ministros eh um absu...
Name: Text, Length: 41409, dtype: object

In [240]:
#vetoriza conjunto completo
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer( )
X_counts = count_vect.fit_transform(X)
X_counts.shape

(41409, 39298)

In [241]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_counts)
X_tf = tf_transformer.transform(X_counts)
X_tf.shape
X_ori = X_tf
#y_ori = df_train.Polarity
X_ori.shape

(41409, 39298)

In [242]:
x_train = X_ori[0:rows_train,:]
x_test = X_ori[rows_train:rows_train+rows_test,:]
x_test.shape


(2354, 39298)

In [243]:
#y_ori = df.Polarity
y_train = df_train.Polarity
y_test = df_test.Polarity
y_test.shape

(2354,)

In [244]:
#classificação
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
model = LogisticRegression(random_state=0)
model.fit(x_train,y_train)
pred_values = model.predict(x_test)     
acc = accuracy_score(pred_values , y_test)
acc

0.7693288020390824

In [245]:
f1_score(y_test, pred_values, average='macro')

0.7497134834595807

In [246]:
precision_score(y_test, pred_values, average='macro')

0.7738435590850982

In [247]:
recall_score(y_test, pred_values, average='macro')

0.7429826212153798

In [248]:
#predição dos não-rotulados
X_label = X_ori[rows_label:rows_label+ rows_unlabel]
pred_values_label= model.predict(X_label) 


In [249]:
#avalia a probabilidade de estar certo
prob_label = model.predict_proba(X_label) 
#np.savetxt('C:/Users/camil/OneDrive/Área de Trabalho/mestrado/Unlabeled/prob.csv', prob_label, delimiter=';')

In [250]:
#seleciona os itens que vão entrar no conjunto de treino
threshold = 0.37
t_size = int(threshold*rows_unlabel) #quanto vai adicionar a cada iteração
rows_train = x_train.shape[0]
rows_train

7060

In [251]:
reliable_pred_favor, reliable_pred_against  = [], []
for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
    #se 1, inclui no favor; senão, no against
    #incluir id, probabilidade e valor predito (1 ou 0)
    (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    

In [252]:
#coloca em ordem decrescente de confiabilidade a partir da probabilidade (coluna 2)
reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)

In [253]:
y= np.asarray(y_train)
index_list=[x for x in range(0, rows_train)] #indices para o novo X a partir do X original
iteractions = 0
position = 0
index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

while iteractions < 0.99:
    iteractions = iteractions + threshold
    for i in range(int(t_size/2)):
        #pega um a favor
        index = reliable_pred_favor[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 1) #empilha o y
        index_labels.remove(index+rows_label)
        #pega um contra
        index = reliable_pred_against[position][0] #descobre a posição no X_label 
        index_list.append(index+rows_label) #inclui na lista de indices
        y = np.append(y, 0) #empilha o y
        index_labels.remove(index+rows_label)
        #anda uma posição nos dois 
        position+=1

    X_train_novo = X_ori[index_list,:]#atualiza o X rotulado com os índices que estão em index_list
    X_label = X_ori[index_labels,:] #pega só os índices que sobraram
    rows_unlabel = X_label.get_shape()[0] #novo número de linhas sem rótulo
    rows_label = X_train_novo.get_shape()[0] + rows_test #linhas com rótulo novas + as de teste que estão guardadas
    X_ori = sparse.vstack([X_train_novo, x_test, X_label]) #empilha treino, teste e unlabelled
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)] #atualiza os unlabelled
    position = 0

   # x_train_novo, x_test_novo, y_train_novo, y_test_novo = train_test_split(
        #X_novo,
        #np.asarray(y),
        #test_size = 0.25,
        #random_state = 123,
       # shuffle = True,
        #stratify = np.asarray(y),
    #)

    model_novo = LogisticRegression(random_state=0)
    model_novo.fit(X_train_novo,y)
    #testa nos conjuntos originais
    pred_values_novo = model_novo.predict(x_test)     
    acc = accuracy_score(pred_values_novo , y_test)
    print("iteraction:",iteractions, "accuracy:", acc)
    
    pred_values_label= model_novo.predict(X_label) 
    prob_label = model.predict_proba(X_label) 
    reliable_pred_favor, reliable_pred_against  = [], []
    for x_id, (prob, pred) in enumerate(zip(prob_label, pred_values_label)):
        (reliable_pred_favor if (pred_values_label[x_id]) else reliable_pred_against).append((x_id, prob_label[x_id,pred_values_label[x_id]], pred_values_label[x_id]))
    
    reliable_pred_favor = sorted(reliable_pred_favor, key=lambda x: x[1], reverse=True)
    reliable_pred_against = sorted(reliable_pred_against, key=lambda x: x[1], reverse=True)    
    
    index_labels=[x for x in range(rows_label, rows_label+ rows_unlabel)]

iteraction: 0.37 accuracy: 0.7429906542056075


C:\Users\camil\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


iteraction: 0.74 accuracy: 0.7276975361087511


IndexError: list index out of range

In [ ]:
len(reliable_pred_against)